# **Health Insurance_Transformation**

## Objectives

* At the end of this stage, I will:
    1. Answer questions from the Extraction stage
    2. Transform data to extract insights
    3. Present visuals for easy analysis

## Inputs

* modified_insurance.csv file is required to run this notebook
* this file extends the work from health_insurance_extraction.ipynb
* the following questions guide the transformation stage:
    1. How does smoking impact insurance charges?
    2. How does region impact charges?
    3. How does gender impact charges?
    4. How does age impact bmi and charges?

## Outputs

* At the end of transformation, there will be DataFrames and graphs that will convey the results

---

# Section 1: Smoking

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Here are the questions that will be guiding me:

    1. How does smoking impact insurance charges?

    2. How does region impact charges?

    3. How does gender impact charges?

    4. How does age impact bmi and charges?

---

In [132]:
df = pd.read_csv("../data/modified_insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


### How does smoking impact insurance charges?

User Story covered: As a business development executive, I want to predict which customer group is most likely to pay high insurance charges so that I can design add-on products and services to optimize value.

Sub-issue: As a business development executive, I want to group customers by their lifestyle habits and key health indicators, so that I can provide recommendations and consultations as a value-added service.

In [14]:
df_chg_sum= df["charges"].sum()
df_chg_sum

17755824.990759

In [21]:
df_smoker = df[df['smoker'] == 'yes']["charges"].sum()
df_smoker

8781763.521839999

I now know that a little less than 50% of the insurance charges are from smokers.  

In [184]:
fig = px.histogram(data_frame=df, x="smoker", y="charges", color="sex", 
                   title="Smoker vs Charges by Sex", barmode="group", 
                   histfunc = "avg", width=1000, height=600)

fig.show()

**Smoker vs Charges by Sex**

Smoking determines the amount of insurance charges. The average of charges paid by females who smoke are 30, 679 and 33, 042 by their male counterparts. 

In contrast, the average of charges paid by females who do not smoke are 8, 762 and 8, 087 by male non-smokers. Let's explore further.

In [185]:
fig = px.box(data_frame=df, x="smoker", y="charges", color="region", 
             title="Smoker vs Charges by Region", width=1000, height=600)

fig.show()

**Smoker vs Charges by Region**

This boxplot informs that smokers pay significantly higher insurance charges compared to non-smokers—almost twice the average insurance charges paid by non-smokers.

### Exploring room for feature engineering

The business will benefit by providing value-added services to customers who are paying high insurance charges. If I create a new group that categorizes respondents based on their lifestyle and health habits, that will help the business categorize customers and tailor services.

In [166]:
def add_risk_categories(df):
    """Categorize individuals into risk categories based on smoking habit and BMI."""

    df['high_risk'] = (df['bmi'] > 30).astype(int)

    df["medium_risk"] = ((df['bmi'] >= 25) & (df["bmi"] <= 30)).astype(int)
    
    df["low-medium_risk"] = ((df['bmi'] >= 17) & (df["bmi"] <= 25)).astype(int)
    
    df["low_risk"] = ((df["smoker"]== "no") & (df["bmi"] < 25)).astype(int)

    return df

df = add_risk_categories(df)
df.to_csv("../data/feature_engineered_insurance.csv", index=False)

Created four new columns categorizing risk. I've built a function around it for ML Engineers to simply apply this to new data, as and when it becomes available. Here's a quick overview:

1. high_risk: bmi is more than 30

2. medium_risk: bmi is between 25 and 30

3. low-medium_risk: bmi is between 17 and 25

4. low_risk: non-smoker and the BMI is less than 25

I leveraged the [NHS BMI INFO](https://www.nhsinform.scot/healthy-living/food-and-nutrition/healthy-eating-and-weight-management/body-mass-index-bmi/) to derive these buckets.

In [167]:
df1 = pd.read_csv("../data/feature_engineered_insurance.csv")
df1

,age,sex,bmi,children,smoker,region,charges,high_risk,medium_risk,low-medium_risk,low_risk
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,0,0
1,18,male,33.770,1,no,southeast,1725.55230,1,0,0,0
2,28,male,33.000,3,no,southeast,4449.46200,1,0,0,0
3,33,male,22.705,0,no,northwest,21984.47061,0,0,1,1
4,32,male,28.880,0,no,northwest,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,1,0,0,0
1334,18,female,31.920,0,no,northeast,2205.98080,1,0,0,0
1335,18,female,36.850,0,no,southeast,1629.83350,1,0,0,0
1336,21,female,25.800,0,no,southwest,2007.94500,0,1,0,0


In [186]:
fig = px.pie(data_frame=df1, names=["high_risk", "medium_risk", "low-medium_risk", 
                                    "low_risk"], 
             values=df1[["high_risk", "medium_risk", "low-medium_risk", 
                         "low_risk"]].sum(), title="Risk categories distribution", 
                         width=1000, height=600)


fig.show()

**Risk categories distribution**

With the risk category feature, the business will be able to build unique services for different groups. For instance, high-risk customers can be recommended a complementary diet and workout plan or they can be redirected to consultation sessions.

---

# Section 2: Region

2. How does region impact charges?

User Stories covered: 

* As a business development executive, I want to predict which customer group is most likely to pay high insurance charges so that I can design add-on products and services to optimize value.

* As an insurance provider, the company wants to determine which region is most likely to show an increased healthcare demand so that we can optimize our investment efforts.

In [178]:
fig = px.box(data_frame=df1, x="region", y="charges", color="region", 
             title="Region vs Charges", width=1000, height=600)

fig.show()

**Region vs Charges**

This boxplot indicates that the charges are widely distributed across regions, however, the median hovers around the 10K mark. We need further visualizations to better understand which region the business can focus on to optimize investment.

In [179]:
categorical_counts = df1.groupby("region")["charges"].sum().reset_index()
fig = px.histogram(data_frame=categorical_counts, x="region", y="charges", 
                   color="region", title="Charges by Region", width=1000, height=600)

fig.show()

**Assumption**: For the sake of this project, I am omitting Southeast region's impact as it consists of significantly larger number of respondents. At the workplace, I would flag this as a bias and inform the data manager/data engineer. I would also recommend best practices to avoid such issues, which I will cover in the Load phase.

To make an impactful recommendation, the analysis needs to consider such nuances. Else, the true potential will still be locked.

**Charges by Region**

Southeast is the highest contributor, however, it also has slightly higher number of respondents compared to other regions. Both Northwest and Southwest generate similar amounts of insurance charges. However, Northeast seems to generate a slightly higher amount of insurance charges. 

We know that smokers contribute more than non-smokers. So, I will probe further before predicting an order of regions that the business can focus on as part of their investment efforts.

In [180]:
fig = px.sunburst(data_frame=df1, path=["region", "smoker"], color= "charges", 
                  hover_name= "charges", 
                  title="Charges by Region and Smoker Status", 
                  color_continuous_scale="tealrose", 
                  labels={"smoker": "smoker status"}, width=1000, height=600)

fig.show()

**Charges by Region and Smoker Status**

This sunburst chart helps further the analysis. Here are some interesting insights, considering that we are aware that smokers contribute more to insurance charges than non-smokers:

1.  As the Southeast region has more number of respondents, it also displays a higher number of respondents(364) and average of charges(14, 735).

2. The Northeast region generates a higher average of insurance charges(13, 406), despite having fewer non-smokers(257) compared to Northwest and Southwest.
    2.a. However, the average of charges generated by 67 smokers is 29, 673. This is lesser than the average of charges from Northwest and Southwest.

3. The average of charges in the Northwest region is 12, 417. The number of smokers is 58 and the average of their charges are 30, 192.

4. The average of charges in the Southwest region is 12, 346. The number of smokers is 58 and the average of their charges are 32, 269.

Based on the above analysis, here is my recommendation:

1. Invest 40% of efforts and resources in the Northeast as that region has a relatively larger share of smokers. Observation from the  Southeast dataset reveals that when you increase the sample size, you are most likely to increase the smoker base as well.
    1.a. Even if the business does not find smokers, the non-smoking population from this region contribute significantly higher than the non-smoking population from other groups.

2. Invest 38% in the Southwest as that region brings in a relatively larger share of charges from smokers. However, we need to be aware that this region contains fewer non-smokers. The room for finding more smokers may be slightly limited and that's why we also reduce the amount of efforts and resources accordingly.

3. Invest 16% each in the Northwest and Southeast. However, I would understand why there is increase sample data from Southeast before initiating fresh investments.

---

# Section 3: Gender

3. How does gender impact charges?

In [181]:
sex_categorical_counts = df1.groupby(["sex", "smoker"])["charges"].mean().reset_index()
fig = px.histogram(data_frame=sex_categorical_counts, x="sex", y="charges", 
                   color="smoker", title="Sex vs Charges", barmode="group", 
                   width=1000, height=600)

fig.show()

**Sex vs Charges**

This chart compares the differences in insurance charges between females and males. Gender does not impact the charges as much as smoking does. 

Interestingly, female non-smokers tend to pay a slightly higher average of charges compared to males. However, male smokers tend to pay significantly higher charges than females.

In [182]:
fig = px.box(data_frame=df1, x="sex", y="charges", color="sex", 
             title="Distribution of Sex by Charges", width=1000, height=600)

fig.show()

**Distribution of Sex by Charges**

This chart compares insurance charges between females and males. The median charges for both genders appear similar, suggesting no major differences in the central tendency. However, there are also more outliers among males, indicating greater variability in charges. This means we need to consider other factors along with gender.

To obtain further value from a gender analysis, I may have to club it with other variables. I will add this to the product backlog and pick it up in the second iteration. 

For the sake of this project, I will progress with understanding how age impacts other variables.

---

# Section 4: Age

4. How does age impact bmi and charges?

User Story covered: As an insurance provider, I want to group customers by age group to build targeted marketing campaigns.

In [122]:
df1["age"].nunique()

47

I will transform the age dataset to visualize better and meet the business requirements.

In [169]:
df1['age_group'] = pd.cut(df['age'], bins=[16, 19, 35, 50, 100], labels=['teen', 'young_adult', 'adult', 'senior'])
df1.value_counts("age_group")

age_group
young_adult    437
adult          408
senior         356
teen           137
Name: count, dtype: int64

In [170]:
df2 = df1.to_csv("../data/feature_engineered_insurance.csv", index=False)
df2 = pd.read_csv("../data/feature_engineered_insurance.csv")
df2

,age,sex,bmi,children,smoker,region,charges,high_risk,medium_risk,low-medium_risk,low_risk,age_group
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,0,0,teen
1,18,male,33.770,1,no,southeast,1725.55230,1,0,0,0,teen
2,28,male,33.000,3,no,southeast,4449.46200,1,0,0,0,young_adult
3,33,male,22.705,0,no,northwest,21984.47061,0,0,1,1,young_adult
4,32,male,28.880,0,no,northwest,3866.85520,0,1,0,0,young_adult
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,1,0,0,0,adult
1334,18,female,31.920,0,no,northeast,2205.98080,1,0,0,0,teen
1335,18,female,36.850,0,no,southeast,1629.83350,1,0,0,0,teen
1336,21,female,25.800,0,no,southwest,2007.94500,0,1,0,0,young_adult


With the new variable, "age_group", the marketing team will have visibility over different groups of customers. This will help them monitor any trends and modify their strategy accordingly.

I have created a "teen" group for respondents aged between 16-19; 
"young_adult" for 20-35;
"adult" covering 35-50; and
"senior" for 50-100

In [183]:
age_categorical_counts = df2.groupby(["age", "sex", "smoker", "age_group"])["charges"].mean().reset_index()
fig = px.histogram(data_frame=age_categorical_counts, x="age_group", y="charges",
                 color="smoker", title="Charges by Age Group and Smoker Status", 
                 barmode="group", histfunc="avg", width=1000, height=600)

fig.show()

**Charges by Age Group and Smoker Status**

This chart compares age groups against the average of charges by smoking habits. Compared to non-smokers, smokers in all age groups bear a hefty insurance price.  

# Section 5: Summary

1. The correlation matrix from the extraction document proves right - Smoking is the determining factor in the amount of insurance charges one pays.

2. Inferred risk categories based on the BMI data available. Utilized the [NHS BMI Info](https://www.nhsinform.scot/healthy-living/food-and-nutrition/healthy-eating-and-weight-management/body-mass-index-bmi/) to slice the values.

3. Categorized age groups into teens, young adult, adult, and senior.

4. Discovered items that require further probing. I will mark them down in the project board for the second iteration.